In [ ]:
import importlib
import numpy as np
import os
import glob
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
% matplotlib inline
import tensorflow as tf
from keras import backend as K
import pickle
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import random
import cv2 as cv
import shutil
import scipy.signal as signal
import scipy
​
import preprocess.filesProc as filesProc
import preprocess.pseudoEcg as pseudoEcg
import preprocess.ecg as Ecg
import dataProc
import opdata
import analysis
import utils
import resUnet3d
import visualize

In [ ]:
elec_pos = np.zeros((25, 2), np.float32)
x_ = np.linspace(93/10, 93-93/10, 5)
for i in range(5):
    elec_pos[i*5:(i+1)*5, 0] = x_
    elec_pos[i*5:(i+1)*5, 1] = np.flip(x_, 0)[i]
print(elec_pos)

In [ ]:
train_path_share = '/mnt/Omer/Project/07.DeepElectrodeMapping/processed_data/20190628_1/train/'
test_path_share = '/mnt/Omer/Project/07.DeepElectrodeMapping/processed_data/20190628_1/test/'
train_path = '/mnt/Omer/Project/07.DeepElectrodeMapping/processed_data/20190710_1/train/'
test_path = '/mnt/Omer/Project/07.DeepElectrodeMapping/processed_data/20190710_1/test/

In [ ]:
z_dist_list = [2, 4, 6, 8, 10]
gnd_pos = np.array([[-93, 46, 10]], np.float32)

In [ ]:
filesProc.getPecgWithRandomElec(os.path.join(train_path_share, 'phie'), train_path, elec_pos, z_dist_list, 8, gnd_pos, (93, 93))
filesProc.getPecgWithRandomElec(os.path.join(test_path_share, 'phie'), test_path, elec_pos, z_dist_list, 8, gnd_pos, (93, 93))

In [ ]:
shutil.rmtree('./temp/train_0710_pecg')
shutil.rmtree('./temp/train_0710_vmem')
shutil.rmtree('./temp/val_0710_pecg')
shutil.rmtree('./temp/val_0710_vmem')

In [ ]:
temp_train_pecg_path = './temp/train_0710_pecg'
filesProc.save3dBlocks(os.path.join(train_path, 'pecg'), 320, temp_train_pecg_path, 3, resize=True, dsize=(32, 32))

In [ ]:
temp_train_vmem_path = './temp/train_0710_vmem'
filesProc.save3dBlocks(os.path.join(train_path_share, 'vmem'), 320, temp_train_vmem_path, 2, resize=True, dsize=(32, 32), repetition=8)

In [ ]:
temp_val_pecg_path = './temp/val_0710_pecg'
temp_val_vmem_path = './temp/val_0710_vmem'

In [ ]:
pecg_blocks_paths = sorted(glob.glob(os.path.join(temp_train_pecg_path, '*.npy')))
vmem_blocks_paths = sorted(glob.glob(os.path.join(temp_train_vmem_path, '*.npy')))
combined = list(zip(pecg_blocks_paths, vmem_blocks_paths))
random.shuffle(combined)
val_blocks_num = int(len(vmem_blocks_paths)*0.2)
pecg_blocks_paths_rdm, vmem_blocks_paths_rdm = zip(*combined)
val_pecg_blocks_paths = pecg_blocks_paths_rdm[0: val_blocks_num]
val_vmem_blocks_paths = vmem_blocks_paths_rdm[0: val_blocks_num]
print(val_pecg_blocks_paths)
print(val_vmem_blocks_paths)

In [ ]:
os.makedirs(temp_val_pecg_path)
os.makedirs(temp_val_vmem_path)

In [ ]:
for i in range(val_blocks_num):
    shutil.move(val_pecg_blocks_paths[i], temp_val_pecg_path)
    shutil.move(val_vmem_blocks_paths[i], temp_val_vmem_path)

In [ ]:
# limit GPU memory usage
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.85
session = tf.Session(config=config)

In [ ]:
model_dir = '/mnt/Omer/Project/07.DeepElectrodeMapping/model/20190710_1'

In [ ]:
os.makedirs(model_dir)

In [ ]:
netg = resUnet3d.getModel(320, 32, 32, 1, 4, 32, 16, 'sigmoid')
netg.summary()

In [ ]:
netg.compile(optimizer=Adam(lr=1e-4), loss='mae', metrics=['mae', 'mse'])

In [ ]:
checkpointer = ModelCheckpoint(os.path.join(model_dir, 'netg.h5'), monitor='val_mean_absolute_error', verbose=1,
                               save_best_only=True, save_weights_only=True, mode='min')
learning_rate = ReduceLROnPlateau('val_mean_absolute_error', 0.1, 10, 1, 'auto', 1e-4, min_lr=1e-8)
early_stopping = EarlyStopping(patience=20, verbose=1)
callbacks = [checkpointer, learning_rate, early_stopping]

In [ ]:
# netg.load_weights('/mnt/Omer/Project/07.DeepElectrodeMapping/model/20190628_1/netg_2.h5')

In [ ]:
pecg_block_train_paths = sorted(glob.glob(os.path.join(temp_train_pecg_path, '*.npy')))
vmem_block_train_paths = sorted(glob.glob(os.path.join(temp_train_vmem_path, '*.npy')))
generator_train = utils.GeneratorCnn3d(pecg_block_train_paths, vmem_block_train_paths, 4, (320, 32, 32, 1))
pecg_block_val_paths = sorted(glob.glob(os.path.join(temp_val_pecg_path, '*.npy')))
vmem_block_val_paths = sorted(glob.glob(os.path.join(temp_val_vmem_path, '*.npy')))
generator_val = utils.GeneratorCnn3d(pecg_block_val_paths, vmem_block_val_paths, 4, (320, 32, 32, 1))

In [ ]:
netg.fit_generator(generator_train, epochs=200, verbose=2, callbacks=callbacks, validation_data=generator_val,
                             use_multiprocessing=True, workers=4, max_queue_size=16)

In [ ]:
with open(os.path.join(model_dir, 'history.pickle'), 'wb') as history_file:
    pickle.dump(netg.history.history, history_file)